In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/im.db')

Importing the BOM database, changing the foreign_gross column to be a numeric type

In [3]:
bom = pd.read_csv('data/bom.movie_gross.csv')
bom['foreign_gross'] = bom.foreign_gross.str.replace('$','').str.replace(',','')
bom['foreign_gross'] = bom.foreign_gross.astype(float)
bom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 132.4+ KB


Importing the tmdb database, no changes necessary

In [4]:
tmdb = pd.read_csv('data/tmdb.movies.csv')
tmdb.rename(columns={'Unnamed: 0': 'id0'}, inplace=True)
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id0                26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


importing The Numbers database and changing the production_budget, domestic_gross and worldwide_gross columns to remove the extraneous characters and change type to float. This will allow us to perform aggregate functions on the data in this column.

In [13]:
tn = pd.read_csv('data/tn.movie_budgets.csv')
tn['production_budget'] = tn.production_budget.str.replace('$','').str.replace(',','').astype(float)
tn['domestic_gross'] = tn.domestic_gross.str.replace('$','').str.replace(',','').astype(float)
tn['worldwide_gross'] = tn.worldwide_gross.str.replace('$','').str.replace(',','').astype(float)
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


Next we will be moving the data into the database contained in the im.db file. This will allow us to do all of our queries and aggregations in sql which is much easier than doing it in pandas.

In [ ]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS movie_budgets(
    id numeric,
    release_date text,
    movie text,
    production_budget numeric,
    domestic_gross numeric,
    worldwide_gross numeric
)
""")
tn.to_sql('movie_budgets', if_exists='replace', connection=conn)

In [ ]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS movie_ratings(
    id0 numeric,
    genre_ids text,
    id numeric,
    original_language text,
    original_title text,
    popularity float64,
    release_date text,
    title text,
    vote_average text,
    vote_count text
)
""")
tmdb.to_sql('movie_ratings', if_exists='replace', connection=conn)

In [ ]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS movie_gross(
    title text,
    studio text,
    domestic_gross numeric,
    foreign_gross numeric,
    year numeric,
)
""")
bom.to_sql('movie_gross', if_exists='replace', connection=conn)